# Data Analysis Migros

### Load libraries, load data, define static variables and set options.

In [113]:
# Import libraries.
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Point, Polygon, shape
from ast import literal_eval

# Load data.
supermarkets = pd.read_csv('supermarket_coordinates.csv')
krais_coord = pd.read_csv('kreis_coordinates.csv')

# Define static variables.
# ef_zh = [(47.388259, 8.468802),
#          (47.418613, 8.480302),
#          (47.425864, 8.502571),
#          (47.438234, 8.575379),
#          (47.350885, 8.568362),
#          (47.337249, 8.522954)]

# Set options.
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 20

In [95]:
# More general method to define whether a coordinate is
# in the polygon or not.
def _isin_polygon(lat, lon, pol):
    
    # Define point.
    point = Point(lat, lon)
    
    return Polygon(pol).contains(point)

In [96]:
# Method to define whether a coordiante is within the 
# polygon of a given list of coordinates.
def isin_polygon(df, poly):
        
    # Empty list.
    res = []
    
    # Loop over all rows.
    for i, v in df.iterrows():
        res.append(_isin_polygon(v['lat'], v['lon'], poly))
    
    return res

In [132]:
# coords: a tuple containing coordinates ordered as (lat, lon).
def merc_proj(coords):

    lat, lon = coords    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + \
        lat * (math.pi/180.0)/2.0)) * scale

    return x, y

In [131]:
# Get an additional columns that indicate whether the stores are 
# located in which Kreis.
for i, d in krais_coord.iterrows():
    header = 'isin_kreis_' + str(d['kreis'])
    supermarkets[header] = isin_polygon(supermarkets[['lat', 'lon']], literal_eval(d['coords']))

# Select only the stores located in Zurich.
sel_stores = supermarkets[(supermarkets['isin_kreis_1'] == True) |
                          (supermarkets['isin_kreis_2'] == True) |
                          (supermarkets['isin_kreis_3'] == True) |
                          (supermarkets['isin_kreis_4'] == True) |
                          (supermarkets['isin_kreis_5'] == True) |
                          (supermarkets['isin_kreis_6'] == True) |
                          (supermarkets['isin_kreis_7'] == True) |
                          (supermarkets['isin_kreis_8'] == True) |
                          (supermarkets['isin_kreis_9'] == True) |
                          (supermarkets['isin_kreis_10'] == True) | 
                          (supermarkets['isin_kreis_11'] == True) |
                          (supermarkets['isin_kreis_12'] == True)]

# Clean the stores by getting rid of "Top Market".
sel_stores = sel_stores[~sel_stores['name'].isin(['Top Market Oerlikon'])]

# Add a column indicating the parent company.
sel_stores['parent'] = np.where(sel_stores['name'].str.contains('Migros|Denner|Alnatura|VOI|Migrolino|M-Outlet'), 'Migros',
                                np.where(sel_stores['name'].str.contains('Coop'), 'Coop',
                                         np.where(sel_stores['name'].str.contains('Aldi'), 'Aldi',
                                                  np.where(sel_stores['name'].str.contains('Lidl'), 'Lidl',
                                                           np.where(sel_stores['name'].str.contains('Spar'), 'Spar', 'Other')))))

# Save to csv.
sel_stores.to_csv('markets_clean.csv')
print(sel_stores[['name', 'parent']])

                             name  parent
4              Coop Bahnhofbrücke    Coop
5                 Migros Wengihof  Migros
13                           Aldi    Aldi
14               Coop Wollishofen    Coop
16             Migros Wollishofen  Migros
21                    Migros City  Migros
38              Coop Hönggermarkt    Coop
39                         Denner  Migros
57            Migros Bahnhof Enge  Migros
65                  Coop Fluntern    Coop
67          Coop Albisriederplatz    Coop
73                    Aldi Suisse    Aldi
74                 Coop Maagplatz    Coop
75             Migros Limmatplatz  Migros
76                Coop Manessehof    Coop
79        Coop City Bellevue Food    Coop
85                   Coop Seefeld    Coop
86              Migros Höschgasse  Migros
87             Migros Stadelhofen  Migros
88               Coop Stadelhofen    Coop
89              Migros Kreuzplatz  Migros
90                Coop Kreuzplatz    Coop
93                Migros Burgwies 